# DIFFERENT WAYS OF VISUALIZING ANNOTATIONS DEPENDING ON THEIR FORMAT

### OBB and Polygonal (obtained with SAM) Annotations

In [ ]:
# OBB AND POLYGONAL ANNOTATIONS
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Input paths
image_path = "/workspace/C_dataset/images/v2_021960.jpg"
label_path = "/workspace/notebook/runs/detect/aabb_B_C/labels/v2_021960.txt"

# Output path
output_dir = "/workspace/Visualise/AABB/aabb_B_C"
os.makedirs(output_dir, exist_ok=True)

# Assign colors per class (RGB)
class_colors = {
    0: (255, 0, 0),
    1: (0, 255, 0),
    2: (0, 0, 255)
}

# Check if image exists
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}")
else:
    # Load and convert image to RGB
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Read annotations
    with open(label_path, "r") as f:
        annotations = f.readlines()

    h, w, _ = image.shape

    # Draw polygons from annotation file
    for ann in annotations:
        data = list(map(float, ann.strip().split()))
        class_id = int(data[0])

        # Convert normalized coords into pixel coords
        points = np.array(data[1:]).reshape(-1, 2)
        points[:, 0] *= w
        points[:, 1] *= h
        points = points.astype(np.int32)

        # Draw polygon with class color
        color = class_colors.get(class_id, (255, 255, 255))
        cv2.polylines(image_rgb, [points], isClosed=True, color=color, thickness=2)

    # Show image with polygons
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("Segmentación YOLO con Colores por Clase")
    plt.show()

    # Save output
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image_bgr)
    print(f"Imagen guardada en: {output_path}")


### XML Annotations

In [ ]:
# XML ANNOTATIONS
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# Input paths
image_path = "/workspace/C_dataset/images/v2_010570.jpg"
xml_path   = "/workspace/B_dataset/Annotations/v1_022306.xml"

# Output path for saving results
output_dir = "/workspace/Visualise/xml_aabb"
os.makedirs(output_dir, exist_ok=True)

# Predefined colors per class
class_colors = {
    "car": (255, 0, 0),   # red
    "bus": (0, 255, 0),   # green
    "truck": (0, 0, 255), # blue
}
DRAW_LABEL_TEXT = True


def parse_voc_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image size if available
    size_el = root.find("size")
    if size_el is not None:
        w = int(size_el.findtext("width", default="0"))
        h = int(size_el.findtext("height", default="0"))
        d = int(size_el.findtext("depth", default="0"))
        size = (w, h, d)
    else:
        size = (0, 0, 0)

    filename = root.findtext("filename", default=None)

    # Collect all object bounding boxes
    objects = []
    for obj in root.findall("object"):
        name = obj.findtext("name", default="object").strip()
        bnd = obj.find("bndbox")
        if bnd is None:
            continue
        xmin = int(float(bnd.findtext("xmin")))
        ymin = int(float(bnd.findtext("ymin")))
        xmax = int(float(bnd.findtext("xmax")))
        ymax = int(float(bnd.findtext("ymax")))
        objects.append({"name": name, "bbox": (xmin, ymin, xmax, ymax)})
    return size, objects, filename


def get_color_for_class(cls_name):
    """Return a consistent color for each class."""
    if cls_name in class_colors:
        return class_colors[cls_name]

    # If not predefined, generate a random color
    rng = np.random.default_rng(abs(hash(cls_name)) % (2**32))
    return tuple(int(x) for x in rng.integers(50, 255, size=3))


def draw_aabb_annotations(image_path, xml_path, output_dir):
    #Draw VOC bounding box annotations on an image and save visualization.
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        return
    if not os.path.exists(xml_path):
        print(f"Error: XML file not found at {xml_path}")
        return

    # Load image
    image_bgr = cv2.imread(image_path)
    if image_bgr is None:
        print(f"Error: Could not read image: {image_path}")
        return
    h, w = image_bgr.shape[:2]

    # Parse XML annotations
    size, objects, fname_in_xml = parse_voc_xml(xml_path)

    # Draw each bounding box
    for obj in objects:
        cls = obj["name"]
        (xmin, ymin, xmax, ymax) = obj["bbox"]

        # Clamp coordinates within image
        xmin = max(0, min(xmin, w - 1))
        xmax = max(0, min(xmax, w - 1))
        ymin = max(0, min(ymin, h - 1))
        ymax = max(0, min(ymax, h - 1))

        color = get_color_for_class(cls)
        cv2.rectangle(image_bgr, (xmin, ymin), (xmax, ymax), color, thickness=2)

        # Optionally draw class label
        if DRAW_LABEL_TEXT:
            label = cls
            (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(image_bgr, (xmin, max(0, ymin - th - baseline - 4)),
                          (xmin + tw + 6, ymin), color, thickness=-1)
            cv2.putText(image_bgr, label, (xmin + 3, ymin - 4),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255),
                        thickness=2, lineType=cv2.LINE_AA)

    # Show image
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("VOC (AABB) Annotations")
    plt.show()

    # Save result
    out_name = os.path.basename(image_path)
    out_path = os.path.join(output_dir, out_name)
    cv2.imwrite(out_path, image_bgr)
    print(f"Imagen guardada en: {out_path}")


# Run visualization
draw_aabb_annotations(image_path, xml_path, output_dir)


### OBB PREDICTIONS

In [ ]:
# OBB PREDICTIONS (OBB ANNOTATIONS + CONFIDENCE)
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Input paths
image_path = "/workspace/C_dataset/images/v2_021960.jpg"
label_path = "/workspace/notebook/runs/detect/aabb_C_C/labels/v2_021960.txt"

# Output path
output_dir = "/workspace/Visualise/AABB/aabb_C_C"
os.makedirs(output_dir, exist_ok=True)

# Colors for each class
class_colors = {
    0: (255, 0, 0),   # red
    1: (0, 255, 0),   # green
    2: (0, 0, 255)    # blue
}

# Text drawing config
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
THICKNESS = 1
TEXT_BG = True 

# Load image
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}")
else:
    image = cv2.imread(image_path)
    if image is None:
        raise RuntimeError(f"No se pudo cargar la imagen: {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]

    # Load annotation file
    with open(label_path, "r") as f:
        annotations = [ln.strip() for ln in f if ln.strip()]

    # Process each annotation line
    for ann in annotations:
        vals = ann.split()
        class_id = int(float(vals[0]))

        # First 8 numbers → polygon points (OBB)
        coord_vals = list(map(float, vals[1:9]))
        points = np.array(coord_vals, dtype=np.float32).reshape(-1, 2)

        # Denormalize to pixel coordinates
        points[:, 0] *= w
        points[:, 1] *= h
        points = points.astype(np.int32)

        # Optional confidence score (if provided in file)
        conf = None
        if len(vals) >= 10:
            try:
                conf = float(vals[9])
            except ValueError:
                conf = None

        # Draw polygon
        color = class_colors.get(class_id, (255, 255, 255))
        cv2.polylines(image_rgb, [points], isClosed=True, color=color, thickness=2)

        # Compute centroid to place text
        centroid = points.mean(axis=0).astype(int)
        tx, ty = int(centroid[0]), int(centroid[1])

        # Label text: class ID + optional confidence
        if conf is not None:
            text = f"{class_id} {conf:.2f}"
        else:
            text = f"{class_id}"

        # Adjust position to keep text inside image
        (tw, th), baseline = cv2.getTextSize(text, FONT, FONT_SCALE, THICKNESS)
        tx = max(0, min(tx, w - tw - 2))
        ty = max(th + 2, min(ty, h - 2))

        # Draw text background for readability
        if TEXT_BG:
            cv2.rectangle(image_rgb, (tx, ty - th - baseline),
                          (tx + tw + 2, ty + 2), (0, 0, 0), -1)

        # Draw text
        cv2.putText(image_rgb, text, (tx + 1, ty), FONT, FONT_SCALE,
                    color, THICKNESS, cv2.LINE_AA)

    # Show visualization
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("OBB YOLO with class and confidence (2 decimals)")
    plt.show()

    # Save visualization
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image_bgr)
    print(f"Image saved at: {output_path}")


### AABB Predictions

In [ ]:
# AABB PREDICTIONS (AABB ANNOTATIONS + CONFIDENCE)
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Input paths
image_path = "/workspace/B_dataset/images/v1_028243.jpg"
label_path = "/workspace/notebook/runs/detect/aabb_C_B/labels/v1_028243.txt"

# Output path
output_dir = "/workspace/Visualise/AABB/aabb_C_B"
os.makedirs(output_dir, exist_ok=True)

# Colors for each class
class_colors = {
    0: (255, 0, 0),   # red
    1: (0, 255, 0),   # green
    2: (0, 0, 255)    # blue
}

# Text drawing settings
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
THICKNESS = 1
TEXT_BG = True  

def yolo_aabb_to_xyxy(xc, yc, w, h, img_w, img_h):
    xc *= img_w
    yc *= img_h
    w  *= img_w
    h  *= img_h

    x1 = int(round(xc - w / 2))
    y1 = int(round(yc - h / 2))
    x2 = int(round(xc + w / 2))
    y2 = int(round(yc + h / 2))
    return x1, y1, x2, y2

# Load image
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}")
else:
    image = cv2.imread(image_path)
    if image is None:
        raise RuntimeError(f"No se pudo cargar la imagen: {image_path}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]

    # Read predictions from YOLO label file
    with open(label_path, "r") as f:
        raw_lines = [ln.strip() for ln in f if ln.strip()]

    parsed = 0 
    skipped = 0 

    # Loop through each prediction line
    for ann in raw_lines:
        vals = ann.split()

        # Expect 5 values (no confidence) or 6 values (with confidence)
        if len(vals) not in (5, 6):
            skipped += 1
            continue

        try:
            class_id = int(float(vals[0]))
            xc = float(vals[1]); yc = float(vals[2])
            bw = float(vals[3]); bh = float(vals[4])
            conf = float(vals[5]) if len(vals) == 6 else None
        except ValueError:
            skipped += 1
            continue

        # Convert normalized AABB to pixel coordinates
        x1, y1, x2, y2 = yolo_aabb_to_xyxy(xc, yc, bw, bh, w, h)

        # Clamp coordinates inside image
        x1 = max(0, min(x1, w-1))
        y1 = max(0, min(y1, h-1))
        x2 = max(0, min(x2, w-1))
        y2 = max(0, min(y2, h-1))

        # Pick color for class
        color = class_colors.get(class_id, (255, 255, 255))

        # Draw bounding box
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness=2)

        # Build label text (class id + optional confidence)
        text = f"{class_id}" if conf is None else f"{class_id} {conf:.2f}"
        (tw, th), baseline = cv2.getTextSize(text, FONT, FONT_SCALE, THICKNESS)

        tx, ty = x1, max(y1, th + 2)
        if TEXT_BG:
            cv2.rectangle(image_rgb,
                          (tx, ty - th - baseline),
                          (tx + tw + 2, ty + 2),
                          (0, 0, 0), -1)

        # Draw label text
        cv2.putText(image_rgb, text, (tx + 1, ty),
                    FONT, FONT_SCALE, color, THICKNESS, cv2.LINE_AA)

        parsed += 1

    # Show visualization
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("YOLO AABB with class and confidence")
    plt.show()

    # Save visualization
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image_bgr)
    print(f"Imagen guardada en: {output_path}")
    print(f"Cajas dibujadas: {parsed} | Líneas ignoradas: {skipped}")
